In [1]:
!pip install influxdb-client

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

import os
import json

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

In [3]:
spark = SparkSession.builder.master('local[*]').getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cf907a95-bb3d-43a4-b6b2-db601b2e8664;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central

In [4]:
cols = ['Time',
 'HMI_FIT101', 'HMI_LIT101', 'HMI_AIT201', 'HMI_AIT202', 'HMI_AIT203', 'HMI_FIT201',
 'HMI_DPIT301', 'HMI_FIT301', 'HMI_LIT301', 'HMI_AIT401', 'HMI_AIT402', 'HMI_FIT401', 
 'HMI_LIT401', 'HMI_AIT501', 'HMI_AIT502', 'HMI_AIT503', 'HMI_AIT504', 'HMI_FIT501',
 'HMI_FIT502', 'HMI_FIT503', 'HMI_FIT504', 'HMI_PIT501', 'HMI_PIT502', 'HMI_PIT503', 
 'HMI_FIT601', 'HMI_MV101', 'HMI_P101', 'HMI_P102', 'HMI_MV201', 'HMI_P201', 'HMI_P202',
 'HMI_P203', 'HMI_P204', 'HMI_P205', 'HMI_P206','HMI_MV301', 'HMI_MV302', 'HMI_MV303',
 'HMI_MV304', 'HMI_P301', 'HMI_P302', 'HMI_P401', 'HMI_P402', 'HMI_P403', 'HMI_P404',
 'HMI_P501', 'HMI_P502', 'HMI_P601', 'HMI_P602', 'HMI_P603', 'HMI_UV401']

fields = [StructField(col_name, StringType(), True) for col_name in cols]
schema = StructType(fields)

In [5]:
inputStream = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "SWAT")\
    .load()
inputStream = inputStream.select(from_json(col("value").cast("string"), schema).alias("data"))
inputStream.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- Time: string (nullable = true)
 |    |-- HMI_FIT101: string (nullable = true)
 |    |-- HMI_LIT101: string (nullable = true)
 |    |-- HMI_AIT201: string (nullable = true)
 |    |-- HMI_AIT202: string (nullable = true)
 |    |-- HMI_AIT203: string (nullable = true)
 |    |-- HMI_FIT201: string (nullable = true)
 |    |-- HMI_DPIT301: string (nullable = true)
 |    |-- HMI_FIT301: string (nullable = true)
 |    |-- HMI_LIT301: string (nullable = true)
 |    |-- HMI_AIT401: string (nullable = true)
 |    |-- HMI_AIT402: string (nullable = true)
 |    |-- HMI_FIT401: string (nullable = true)
 |    |-- HMI_LIT401: string (nullable = true)
 |    |-- HMI_AIT501: string (nullable = true)
 |    |-- HMI_AIT502: string (nullable = true)
 |    |-- HMI_AIT503: string (nullable = true)
 |    |-- HMI_AIT504: string (nullable = true)
 |    |-- HMI_FIT501: string (nullable = true)
 |    |-- HMI_FIT502: string (nullable = true)
 |    |-- HMI_FIT503: st

In [6]:
data = inputStream.select("data.*")

In [7]:
def processBatch(df, id):
    # https://spark.apache.org/docs/2.3.0/sql-programming-guide.html
    cols = list(set(df.columns) - {'Time'})
    for row in df.rdd.collect():
        timestamp = row['Time']
        values = df.select(cols)
        values.select('HMI_LIT101').show()

In [8]:
# 3. Output data: show result in the console
query = (data
         .writeStream
         .outputMode("append")
         .foreachBatch(processBatch)
         .start())

query.awaitTermination()

+----------+
|HMI_LIT101|
+----------+
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
+----------+



KeyboardInterrupt: 